# NB

### In order for the program to behave correctly, the main logi should be run without executing tests.
for convenience, we designed tests to use the same ontologyes created to run the program, hence, their ezecution could pollute the results. We commented them in this notebook such that you could run all the cells without experiencing problems.

our suggestion, then, is to run tests and the main logic separately

# STEP 1 - creation of the java ontology

In [1]:
import sys

sys.path.append('/usr/local/lib/python3.8/site-packages')

from os import path as Path
import ast
from owlready2 import *
import javalang.tree
import rdflib
import rdflib.plugins.sparql as sq

# relative path of the subject code folder
chess_path              = './resources/android-chess/app/src/main/java/jwtc/chess/'
# relative path of the java tree structure
treePy_path             = './resources/tree.py'
# relative path of our populated ontology
populated_ontology_path = './computed/tree2.owl'
# relative path of our empty ontology
ontology_path           = './computed/tree.owl'
# relative path for queries
queries_path           = './queries/'

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
class Visitor(ast.NodeVisitor):

    def __init__(self, ontology):
        self.ontology = ontology

    def generic_visit(self, node):
        ast.NodeVisitor.generic_visit(self, node)
        with self.ontology as onto:
            if type(node) == ast.ClassDef:
                for obj in node.bases:
                    if obj.id == "Node":
                        types.new_class(node.name, (Thing,))
                    else:
                        types.new_class(node.name, (onto[obj.id],))

            elif type(node) == ast.Assign:
                for el in node.value.elts:
                    if el.s == "body" or el.s == "parameters":
                        types.new_class(el.s, (ObjectProperty,))
                    else:
                        types.new_class(
                            "jname" if el.s == "name" else el.s,
                            (DataProperty,))


In [3]:
def create_ontology(path):
    with open(path, "r") as source:
        tree = ast.parse(source.read())

    ontology = get_ontology("http://Java_Ontology/JavaTree.owl")

    visitor = Visitor(ontology)
    visitor.visit(tree)
    if os.path.exists(ontology_path):
        os.remove(ontology_path)
    ontology.save(ontology_path, format="rdfxml")
    
create_ontology("./resources/tree.py")

## UNIT TEST

In [4]:
# onto = get_ontology(ontology_path).load()
# cd = onto["ClassDeclaration"]

# assert cd.name == "ClassDeclaration"
# assert len(cd.is_a) == 1
# assert cd.is_a[0].name == 'TypeDeclaration'

In [5]:
# onto = get_ontology(ontology_path).load()
# cd = onto["MethodDeclaration"]

# assert cd.name == "MethodDeclaration"
# assert len(cd.is_a) == 2
# assert cd.is_a[0].name == 'Member'
# assert cd.is_a[1].name == 'Declaration'

In [6]:
# onto = get_ontology(ontology_path).load()
# cd = onto["FieldDeclaration"]

# assert cd.name == "FieldDeclaration"
# assert len(cd.is_a) == 2
# assert cd.is_a[0].name == 'Member'
# assert cd.is_a[1].name == 'Declaration'

In [7]:
# onto = get_ontology(ontology_path).load()
# cd = onto["ConstructorDeclaration"]

# assert cd.name == "ConstructorDeclaration"
# assert len(cd.is_a) == 2
# assert cd.is_a[0].name == 'Declaration'
# assert cd.is_a[1].name == 'Documented'

In [8]:
# onto = get_ontology(ontology_path).load()
# cd = onto["VariableDeclarator"]

# assert cd.name == "VariableDeclarator"
# assert len(cd.is_a) == 1

## STATS

`Axiom`							: 	4577
	
`Logical axiom count`			: 	3073
	
`Declaration axioms count`		: 	1504
	
`Class count`					: 	  78
	
`Object property count`			: 	   2
	
`Data property count`			: 	  65
	
`Individual count`				: 	1360

`SubClassOf`					: 	  83

`ClassAssertion`				: 	1360
	
`ObjectPropertyAssertion`		: 	1347
	
`DataPropertyAssertion`			: 	 283



# STEP 2-3 - populating the ontology

In [9]:
def append_fields(class_declaration, fields, ontology):
    for field in fields:
        o_fd = ontology['FieldDeclaration']()
        o_fd.jname = [field.name]
        class_declaration.body.append(o_fd)

def append_method(class_declaration, method, ontology):
    o_md = ontology['MethodDeclaration']()
    o_md.jname = [method.name]
    append_statements(o_md, method, ontology)
    append_parameters(o_md, method, ontology)
    class_declaration.body.append(o_md)
    
def append_constructor(class_declaration, method, ontology):
    o_con = ontology['ConstructorDeclaration']()
    o_con.jname = [method.name]
    append_statements(o_con, method, ontology)
    append_parameters(o_con, method, ontology)
    class_declaration.body.append(o_con)

def append_statements(md, method, ontology):
    for _, statement in method.filter(javalang.tree.Statement):
        if type(statement) != javalang.tree.Statement:
            s_type = statement.__class__.__name__
            s = ontology[s_type]()
            md.body.append(s)

def append_parameters(md, method, ontology):
    for _, statement in method.parameters:
        fp = ontology['FormalParameter']()
        md.parameters.append(fp)

In [10]:
def extract_class_declaration(source):
    class_declarations = []
    for file in os.listdir(source):
        if file.endswith('.java'):
            f_path = os.path.join(source, file)
            with open(f_path) as j_file:
                ast = javalang.parse.parse(j_file.read())
                for _, node in ast.filter(javalang.tree.ClassDeclaration):
                    class_declarations.append(node) 
    return class_declarations

def addClasses(ontology, class_declarations):
    for java_class in class_declarations:
        ontology_cd = ontology['ClassDeclaration']()
        ontology_cd.jname = [java_class.name]
        [append_fields(ontology_cd, f.declarators, ontology) for f in java_class.body if type(f) == javalang.tree.FieldDeclaration]
        [append_method(ontology_cd, m, ontology) for m in java_class.body if type(m) == javalang.tree.MethodDeclaration]
        [append_constructor(ontology_cd, c, ontology) for c in java_class.body if type(c) == javalang.tree.ConstructorDeclaration]
        
def add_instances(ontology_path, source_path):
    class_declarations = extract_class_declaration(source_path)
    ontology = get_ontology(ontology_path).load()
    with ontology:
        addClasses(ontology, class_declarations)
    if os.path.exists(populated_ontology_path):
        os.remove(populated_ontology_path)
    ontology.save(populated_ontology_path, format="rdfxml")
    


In [11]:
add_instances(ontology_path, chess_path)

## Stats


ClassDeclaration		:	11 

MethodDeclaration		:	152 

FieldDeclaration		:	105 

ConstructorDeclaratio	:	6 

FormalParameter			:	165 

AssertStatement			:	0 

BlockStatement			:	143 

BreakStatement			:	23 

CatchClause				:	8 

ContinueStatement		:	4 

DoStatement				:	2 

ForStatement			:	6 

IfStatement				:	125 

ReturnStatement			:	106 

StatementExpression		:	446 

SwitchStatement			:	8 

SynchronizedStatemen	:	1 

ThrowStatement			:	15 

TryStatement			:	8 

WhileStatement			:	10

## UNIT TESTS 2

In [12]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class A { int x, y; }")

# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   

# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)
# onto.save(populated_ontology_path, format="rdfxml")

# a = onto['ClassDeclaration'].instances()[-1]

# assert a.body[0].is_a[0].name == 'FieldDeclaration'
# assert a.body[0].jname[0] == 'x'
# assert a.body[1].is_a[0].name == 'FieldDeclaration'
# assert a.body[1].jname[0] == 'y'
# assert a.jname[0] == 'A'

In [13]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse('class Main { int sum; Main() { this(5, 2); } Main(int arg1, int arg2) { this.sum = arg1 + arg2; } void display() { System.out.println("Sum is: " + sum); } public static void main(String[] args) { Main obj = new Main(); obj.display(); } }')
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)
# onto.save(populated_ontology_path, format="rdfxml")

# a = onto['ClassDeclaration'].instances()[-1]
    
# assert a.jname[0] == 'Main'
# assert a.body[0].is_a[0].name == 'FieldDeclaration'
# assert a.body[0].jname[0] == 'sum'
# assert a.body[1].is_a[0].name == 'MethodDeclaration'
# assert a.body[1].jname[0] == 'display'
# assert a.body[2].is_a[0].name == 'MethodDeclaration'
# assert a.body[2].jname[0] == 'main'
# assert a.body[3].is_a[0].name == 'ConstructorDeclaration'
# assert a.body[3].jname[0] == 'Main'
# assert a.body[4].is_a[0].name == 'ConstructorDeclaration'
# assert a.body[4].jname[0] == 'Main'

# assert a.body[4].parameters[0].is_a[0].name == 'FormalParameter'
# assert a.body[4].parameters[1].is_a[0].name == 'FormalParameter'

## UNIT TESTS 3

In [14]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse('class A { int f() { return 0; } }')
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)
# onto.save(populated_ontology_path, format="rdfxml")

# a = onto['ClassDeclaration'].instances()[-1]
# assert a.body[0].body[0].is_a[0].name == 'ReturnStatement'

# BAD SMELLS


In [15]:
def log(header, rows, logFunc):
    out = open(queries_path + "/" + header + ".txt", "w")
    for row in rows:
        out.write(logFunc(row))
    out.close()
    
if not os.path.exists(queries_path):
        os.makedirs(queries_path)
        
graph = default_world.as_rdflib_graph()

In [16]:
longMethods = sq.prepareQuery(
 """SELECT ?className ?methodName ?statements (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
                ?method tree:body ?statement .
                ?statement a/rdfs:subClassOf* tree:Statement .
            } GROUP BY ?method
            HAVING (COUNT(?statement) >= 20)
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

def longMethodsLog(row):
    return "CLASS: " + row.className + "\t METHOD: " + row.methodName + "\t STATEMENTS COUNT: " + row.tot + "\n"

In [17]:
longConstructors = sq.prepareQuery(
 """SELECT ?className ?constructorName ?statements (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?constructor .
                ?constructor a tree:ConstructorDeclaration .
                ?constructor tree:jname ?constructorName .
                ?constructor tree:body ?statement .
                ?statement a/rdfs:subClassOf* tree:Statement .
            } GROUP BY ?constructor
            HAVING (COUNT(?statement) >= 20)
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

def longConstructorsLog(row):
    return "CLASS: " + row.className + "\t CONSTRUCTOR: " + row.methodName + "\t STATEMENTS COUNT: " + row.tot + "\n"

In [18]:
largeClasses = sq.prepareQuery(
 """SELECT ?className ?methods (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
            } GROUP BY ?className
            HAVING (COUNT(?method) >= 10)
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

def largeClassesLog(row):
    return "CLASS: " + row.className + "\t METHODS COUNT: " + row.tot + "\n" 

In [19]:
methodWithSwitch = sq.prepareQuery(
 """SELECT ?className ?methodName (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
                ?method tree:body ?statement .
                ?statement a tree:SwitchStatement .
            } GROUP BY ?method
            HAVING (COUNT(?method) >= 1)
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

def methodWithSwitchLog(row):
    return "CLASS: " + row.className + "\t METHOD: " + row.methodName + "\t SWITCH COUNT: " + row.tot + "\n"

In [20]:
constructorWithSwitch = sq.prepareQuery(
 """SELECT ?className ?constructorName (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?constructor .
                ?constructor a tree:ConstructorDeclaration .
                ?constructor tree:jname ?constructorName .
                ?constructor tree:body ?statement .
                ?statement a tree:SwitchStatement .
            } GROUP BY ?constructor
            HAVING (COUNT(?constructor) >= 1)
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

def constructorWithSwitchLog(row):
    return "CLASS: " + row.className + "\t CONSTRUCTOR: " + row.methodName + "\t SWITCH COUNT: " + row.tot + "\n"

In [21]:
methodWithLongParameterList = sq.prepareQuery(
 """SELECT ?className ?methodName ?parameter (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
                ?method tree:parameters ?parameter .
                ?parameter a tree:FormalParameter .
            } GROUP BY ?method
            HAVING (COUNT(?parameter) >= 5)
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

def MethodWithLongParameterListLog(row):
    return "CLASS: " + row.className + "\t METHOD: " + row.methodName + "\t PARAMETERS COUNT: " + row.tot + "\n"

In [22]:
contructorWithLongParameterList = sq.prepareQuery(
 """SELECT ?className ?constructorName ?parameter (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?constructor .
                ?constructor a tree:ConstructorDeclaration .
                ?constructor tree:jname ?constructorName .
                ?constructor tree:parameters ?parameter .
                ?parameter a tree:FormalParameter .
            } GROUP BY ?constructor
            HAVING (COUNT(?parameter) >= 5)
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

def contructorWithLongParameterListLog(row):
    return "CLASS: " + row.className + "\t CONSTRUCTOR: " + row.constructorName + "\t PARAMETERS COUNT: " + row.tot + "\n"

In [23]:
dataClass = sq.prepareQuery(
 """SELECT ?className (COUNT(*)AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
                FILTER (regex(?methodName, "set.*") || regex(?methodName, "get.*"))
            } GROUP BY ?className
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

retriveAllMethods = sq.prepareQuery(
    """SELECT ?className (COUNT(*) AS ?tot) WHERE {
                ?class a tree:ClassDeclaration .
                ?class tree:jname ?className .
                ?class tree:body ?method .
                ?method a tree:MethodDeclaration .
                ?method tree:jname ?methodName .
            } GROUP BY ?className
        """,
 initNs = { "tree": "http://Java_Ontology/JavaTree.owl#" })

In [24]:
res = graph.query(longMethods)
log("LongMethods", res, longMethodsLog)

res = graph.query(longConstructors)
log("LongConstructors", res, longConstructorsLog)
    
res = graph.query(largeClasses)
log("LargeClasses", res, largeClassesLog)

res = graph.query(contructorWithLongParameterList)
log("ConstructorWithLongParameterList", res, contructorWithLongParameterListLog)

res = graph.query(methodWithLongParameterList)
log("MethodWithLongParameterList", res, MethodWithLongParameterListLog)

res = graph.query(constructorWithSwitch)
log("ConstructorWithSwitch", res, constructorWithSwitchLog)

res = graph.query(methodWithSwitch)
log("MethodsWithSwitch", res, methodWithSwitchLog)

resGS = graph.query(dataClass)
resAll = graph.query(retriveAllMethods)

out = open(queries_path + "/DataClasses.txt", "w")
for rowGS in resGS:
    for rowA in resAll:
        if rowGS.className == rowA.className:
            if rowGS.tot == rowA.tot:
                out.write("CLASS: " + rowGS.className + "\t GETTERS/SETTERS: " + rowGS.tot + "\n")
out.close()

# UNIT TESTS

In [ ]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class A { int f(int x) { x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++; } }")
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)

# onto.save(file="./test/tmp.owl", format="rdfxml")

# g = rdflib.Graph()
# g.load("./test/tmp.owl")
# res = g.query(longMethods)
# assert len(res) == 1

In [ ]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class MyClaas { public MyClaas(int x) { x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++; } }")
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)

# onto.save(file="./test/tmp.owl", format="rdfxml")

# g = rdflib.Graph()
# g.load("./test/tmp.owl")
# res = g.query(longConstructors)
# assert len(res) == 1

In [ ]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class MyClaas { public A(int x) { x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++;x++; } public void AA(){} public void B(){} public void C(){} public void D(){} public void E(){} public void F(){} public void G(){} public void H(){} public void I(){} public void J(){} public void L(){} public void K(){} public void M(){} }")
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)

# onto.save(file="./test/tmp.owl", format="rdfxml")

# g = rdflib.Graph()
# g.load("./test/tmp.owl")
# res = g.query(largeClasses)
# assert len(res) == 1

In [ ]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class MyClaas { public void A(int x){ int day = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)

# onto.save(file="./test/tmp.owl", format="rdfxml")

# g = rdflib.Graph()
# g.load("./test/tmp.owl")
# res = g.query(methodWithSwitch)
# assert len(res) == 1

In [ ]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class MyClaas { public MyClaas(int x){ int day = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)

# onto.save(file="./test/tmp.owl", format="rdfxml")

# g = rdflib.Graph()
# g.load("./test/tmp.owl")
# res = g.query(constructorWithSwitch)
# assert len(res) == 1

In [ ]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class MyClaas { public void A(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ int day = 4; int day1 = 4; int d2ay = 4; int day3 = 4;  int d22ay = 4; int d2234ay = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)

# onto.save(file="./test/tmp.owl", format="rdfxml")

# g = rdflib.Graph()
# g.load("./test/tmp.owl")
# res = g.query(methodWithLongParameterList)
# assert len(res) == 1

In [ ]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class MyClaas { public MyClaas(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ int day = 4; int day1 = 4; int d2ay = 4; int day3 = 4;  int d22ay = 4; int d2234ay = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)

# onto.save(file="./test/tmp.owl", format="rdfxml")

# g = rdflib.Graph()
# g.load("./test/tmp.owl")
# res = g.query(methodWithLongParameterList)
# assert len(res) == 1

In [ ]:
# onto = get_ontology(ontology_path).load()
# tree = javalang.parse.parse("class MyClaas { public void setInt(int xw,int xww,int xwww,int xe,int xee,int xeeeee,int xf){ int day = 4; int day1 = 4; int d2ay = 4; int day3 = 4;  int d22ay = 4; int d2234ay = 4; switch (day) {case 1: System.out.println(day); break; case 2: System.out.println(day);  break;} }}")
# cds = []
# for _, node in tree.filter(javalang.tree.ClassDeclaration):
#     cds.append(node)   
    
# with onto:
#     addClasses(onto, cds)
# if os.path.exists(populated_ontology_path):
#     os.remove(populated_ontology_path)

# onto.save(file="./test/tmp.owl", format="rdfxml")

# g = rdflib.Graph()
# g.load("./test/tmp.owl")
# res = g.query(dataClass)
# assert len(res) == 1